In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error


z = pd.read_csv(r"C:\Users\gromo\OneDrive\Documents\transactions_diy.csv")

# загружаем датасет

In [2]:
# смотрим датасет

z

,TrDte,BCode,ClientID,Item,ItemGroup,Quantity,Amount
0,01.09.2017,code000000001,client13166,sku8444,Скобяные изделия,1,29
1,01.09.2017,code000000001,client13166,sku12545,Оборудование для сада и дачи,1,329
2,01.09.2017,code000000001,client13166,sku3391,Инструменты,1,169
3,01.09.2017,code000000001,client13166,sku20444,Инструменты,2,578
4,01.09.2017,code000000002,client1239,sku29959,Скобяные изделия,1,329
...,...,...,...,...,...,...,...
1008683,31.10.2019,code000290228,client25258,sku5837,Скобяные изделия,1,89
1008684,31.10.2019,code000290228,client25258,sku26161,Стойматериалы,2,598
1008685,31.10.2019,code000290228,client25258,sku20658,Сантехника,1,159
1008686,31.10.2019,code000290229,client22472,sku16069,Столярные изделия,1,85


In [3]:
# смотрим какие типы данных

z.dtypes

TrDte        object
BCode        object
ClientID     object
Item         object
ItemGroup    object
Quantity      int64
Amount        int64
dtype: object

In [4]:
# замечаем, что TrDte - object


In [5]:
z['Month'] = z['TrDte']
z['Month'] = pd.to_datetime(z['Month'], dayfirst=True)
z['Month'] = z['Month'].dt.month

z['Year'] = z['TrDte']
z['Year'] = pd.to_datetime(z['Year'], dayfirst=True)
z['Year'] = z['Year'].dt.year


In [6]:
# Для первого задания мы создадим отдельный датафрейм с сортировкой ItemGroup по сумме Quantity
# Полученный результат отсортируем по столбцу Amount по убыванию
# Выведем строку с индексом 0 и поймем самую популярную группу (начало нумирации - 0)

z1 = z.groupby(['ItemGroup']).sum(['Quantity'])
z1 = z1.sort_values(by=['Quantity'], ascending=[0])
z1.iloc[0]

Quantity       449237
Amount       80901067
Month          708001
Year        205711431
Name: Стойматериалы, dtype: int64

In [7]:
z1

,Quantity,Amount,Month,Year
ItemGroup,,,,
Стойматериалы,449237,80901067,708001,205711431
Электротовары,229897,30454380,711448,205564212
Комнатные растения и цветы,229185,13896462,758437,326751549
Столярные изделия,223384,51964606,355534,102118196
Лаки и краски,208556,50376538,1021041,297926033
Скобяные изделия,202714,22956800,670609,195746897
Оборудование для сада и дачи,200599,50765598,707150,245610248
Сантехника,114636,28714817,435428,125482917
Кафель и плитка,104898,42534756,91576,27456148


In [8]:
# зная название группы просто выведем df по фильтру группы и посмотрим длину матрицы

z['ItemGroup'].to_string()
z2 = z[(z['ItemGroup'] == 'Стойматериалы')]
z2.shape[0]

101914

In [9]:
# Расчет MARE по Освещению

z3 = z
z3['sum_amount'] = z3.groupby(['ItemGroup', 'Year', 'Month'])['Amount'].transform('sum')
z3['sum_quantity'] = z3.groupby(['ItemGroup', 'Year', 'Month'])['Quantity'].transform('sum')
z3 = z3.drop_duplicates(subset=['ItemGroup', 'Year', 'Month'])
z3 = z3.drop(['TrDte', 'BCode', 'ClientID',  'Item', 'Amount'], axis=1)
z3 = z3.sort_values(by=['ItemGroup', 'Year', 'Month'], ascending=[True, True, True])
z3['roling_sum'] = (z3['sum_quantity'].rolling(window=4).sum() - z3['sum_quantity']) / 3
z3['for_mare'] = abs(z3['sum_quantity'] - z3['roling_sum']) / z3['sum_quantity']
z3_1 = z3
z3 = z3[(z3['ItemGroup'] == 'Освещение')]
z3['for_mare'].sum() / z3.shape[0]

0.6720638359600616

In [10]:
# "Минимальное" МAPE + roling sum по нему

z4 = z
z4['sum_amount'] = z4.groupby(['ItemGroup', 'Year', 'Month'])['Amount'].transform('sum')
z4['sum_quantity'] = z4.groupby(['ItemGroup', 'Year', 'Month'])['Quantity'].transform('sum')
z4 = z4.drop_duplicates(subset=['ItemGroup', 'Year', 'Month'])
z4 = z4.drop(['BCode', 'ClientID',  'Item', 'Amount'], axis=1)
z4 = z4.sort_values(by=['ItemGroup', 'Year', 'Month'], ascending=[True, True, True])
z4['roling_sum'] = (z4['sum_quantity'].rolling(window=4).sum() - z4['sum_quantity']) / 3 
z4 = z4[(z4['Year'] == 2019) | (z4['Year'] == 2018) | ((z4['Year'] == 2017) & (z4['Month'] == 12))]
z4['for_mare'] = abs(z4['sum_quantity'] - z4['roling_sum']) / z4['sum_quantity']
z4['MAPE'] = z4.groupby(['ItemGroup'])['for_mare'].transform('sum') / z4.groupby(['ItemGroup'])['TrDte'].transform('count')
z4 = z4[(z4['Year'] == 2019) & (z4['Month'] == 10)]
z4 = z4.sort_values(by=['MAPE'], ascending=[True])

z4

,TrDte,ItemGroup,Quantity,Month,Year,sum_amount,sum_quantity,roling_sum,for_mare,MAPE
952669,01.10.2019,Столярные изделия,10,10,2019,2801761,12133,12176.000000,0.003544,0.134455
952584,01.10.2019,Электротовары,2,10,2019,2026472,14757,13740.333333,0.068894,0.152711
952580,01.10.2019,Скобяные изделия,1,10,2019,1586599,12446,12422.666667,0.001875,0.158221
952579,01.10.2019,Лаки и краски,2,10,2019,3121456,13439,14112.666667,0.050128,0.161454
952593,01.10.2019,Инструменты,1,10,2019,3761391,4680,4829.000000,0.031838,0.165294
952592,01.10.2019,Обои,1,10,2019,1338714,1238,1591.333333,0.285407,0.181800
952585,01.10.2019,Сантехника,2,10,2019,1774923,7870,7592.000000,0.035324,0.183464
952618,01.10.2019,Ванная комната,1,10,2019,2546617,964,1151.000000,0.193983,0.183739
952595,01.10.2019,Стойматериалы,1,10,2019,5918653,33845,29079.000000,0.140818,0.204124
952880,01.10.2019,Кафель и плитка,2,10,2019,1823792,3934,4892.000000,0.243518,0.205453


In [11]:
# Для определения кол-ва групп используем метод .cut
# для границ вехней границы по группа укажем макс значение по TrDte

z5 = z
z5 = z5.groupby(['ClientID'])['TrDte'].nunique().reset_index()
trdte_max = z5['TrDte'].max()
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, trdte_max]
z5['quantile'] = pd.cut(z5['TrDte'], bins=bins, right=True)
z5['quantile'].value_counts()

quantile
(0, 1]       12859
(1, 2]        6451
(14, 174]     4517
(2, 3]        4222
(3, 4]        3064
(4, 5]        2409
(5, 6]        1941
(6, 7]        1547
(7, 8]        1284
(8, 9]        1069
(9, 10]        854
(10, 11]       784
(11, 12]       709
(12, 13]       550
(13, 14]       486
Name: count, dtype: int64

In [12]:
# В датасете считаем сумму клиента по заказам по товарным группам в оконной функции
# Убираем дубли по клиентам и названиям групп товаров
# сортируем датафрейм по убыванию клиента, суммы по заказам и группе товаров
# считаем кумулянту в оконной функции по столбцу cum_sum
# убираем суммы, не равные 1

z6 = z
z6['sum_amount'] = z6.groupby(['ClientID', 'ItemGroup'])['Amount'].transform('sum')
z6 = z6.drop_duplicates(subset=['ClientID', 'ItemGroup'])
z6 = z6.sort_values(by=['ClientID', 'sum_amount', 'ItemGroup'], ascending=[True, False, True])
z6['rank'] = 1
z6['cum_rank'] = z6.groupby(['ClientID'])['rank'].transform('cumsum')
z8 = z6
z8 = z8.drop(['TrDte', 'BCode', 'Amount', 'Month', 'Year', 'Item', 'Quantity', 'sum_quantity', 'rank'], axis=1)
z6[(z6['cum_rank'] == 1) & (z6['ItemGroup'] == 'Освещение')].shape[0]

2366

In [13]:
# для решения задания по кол-ву пустых значений во 2 группе нужно сделать матрицу, шириной 3
# в 2 датафреймах считаем по индексам 1 и 2 две группы для пользователя
# объединяем с исходным, убираем лишние столбцы

z8_1_for_merge = z8.drop(['sum_amount'], axis=1)
z8_1_for_merge = z8_1_for_merge[z8_1_for_merge['cum_rank'] == 1]
z8_1_for_merge = z8_1_for_merge.rename({'ItemGroup': '1st_group'}, axis=1).drop(['cum_rank'], axis=1)
z8_2_for_merge = z8.drop(['sum_amount'], axis=1)
z8_2_for_merge = z8_2_for_merge[z8_2_for_merge['cum_rank'] == 2]
z8_2_for_merge = z8_2_for_merge.rename({'ItemGroup': '2nd_group'}, axis=1).drop(['cum_rank'], axis=1)

z8 = z8.merge(z8_1_for_merge, how='left', on=['ClientID'])
z8 = z8.merge(z8_2_for_merge, how='left', on=['ClientID'])

z8 = z8.drop(['ItemGroup', 'cum_rank', 'sum_amount'], axis=1)
z8 = z8.drop_duplicates(subset=['ClientID'])

# z8 - матрица из 3 столбцов - клиент, 1 группа, 2 группа
# если ошибка - нужно переактивировать предыдущий код

In [14]:
# Ответ на предыдущее задание 
z8['2nd_group'].isna().sum()

6586

In [15]:
z9 = z
z9['sum_quantity'] = z9.groupby(['ClientID', 'ItemGroup'])['Quantity'].transform('sum')
z9 = z9.drop_duplicates(subset=['ClientID', 'ItemGroup'])
z9 = z9.sort_values(by=['ClientID', 'sum_quantity', 'ItemGroup'], ascending=[True, False, True])
z9['rank'] = 1
z9['cum_rank'] = z9.groupby(['ClientID'])['rank'].transform('cumsum')

z9 = z9.drop(['TrDte', 'BCode', 'Amount', 'Month', 'Year', 'Item', 'Quantity', 'sum_amount', 'rank'], axis=1)

z9_1_for_merge = z9.drop(['sum_quantity'], axis=1)
z9_1_for_merge = z9_1_for_merge[z9_1_for_merge['cum_rank'] == 1]
z9_1_for_merge = z9_1_for_merge.rename({'ItemGroup': '1st_group'}, axis=1).drop(['cum_rank'], axis=1)
z9_2_for_merge = z9.drop(['sum_quantity'], axis=1)
z9_2_for_merge = z9_2_for_merge[z9_2_for_merge['cum_rank'] == 2]
z9_2_for_merge = z9_2_for_merge.rename({'ItemGroup': '2nd_group'}, axis=1).drop(['cum_rank'], axis=1)

z9 = z9.merge(z9_1_for_merge, how='left', on=['ClientID'])
z9 = z9.merge(z9_2_for_merge, how='left', on=['ClientID'])

z9 = z9.drop(['ItemGroup', 'cum_rank', 'sum_quantity'], axis=1)
z9 = z9.drop_duplicates(subset=['ClientID'])

z9[z9['1st_group'] == 'Освещение'].shape[0]

1034